#  <center> Homework 3 </center>
## <center> Daxi Cheng, Tomas Fernandez, Yanbin Kang </center> 

In [1]:
from pandas import Series, DataFrame
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Question 1: Read in data
* We used the tab as a seperator.

In [2]:
gold = pd.read_csv('gold.txt',
                      sep='\t',      # use tab separator
                      header=None,   # do not use the first line as a header
                      names=['url', 'category'])
labels = pd.read_csv('labels.txt',
                      sep='\t',      # use tab separator
                      header=None,   # do not use the first line as a header
                      names=['turk','url', 'category'])

# Question 2:  Split into two DataFrames
* We created a mask to measure if it is in gold

In [3]:
mask_in_gold= labels['url'].isin(gold['url'])
labels_on_gold=labels[mask_in_gold].drop_duplicates()
labels_unknown=labels[~mask_in_gold].drop_duplicates()

# Question 3: Compute accuracies of turks
* Here we compute the accuracy rate of each turk by comparing their results to the gold set using a conditional assignment.

In [4]:
turk_gold_df=labels_on_gold.merge(gold, left_on='url', right_on='url',suffixes=['-turk','-gold'])
turk_gold_df['correct']=turk_gold_df['category-gold']==turk_gold_df['category-turk']

In [5]:
rate_correctness=DataFrame(turk_gold_df.groupby(['turk'])['correct'].mean())
rate_count=DataFrame(turk_gold_df.groupby(['turk'])['category-turk'].count())
rate_correctness['turk']=rate_correctness.index
rate_count['turk']=rate_count.index

In [6]:
rater_goodness=pd.merge(rate_correctness,rate_count)
rater_goodness['number of ratings']=rater_goodness['category-turk']
rater_goodness['average correctness']=rater_goodness['correct']
rater_goodness.index=rater_goodness['turk']
rater_goodness=rater_goodness[['number of ratings','average correctness']]


# Question 4: Odds ratios
* Here we calculate the odds ratios.

In [7]:
def count_odd(p):
    odd=p*1.0/(1.001-p)
    return odd
rater_goodness['odds']=rater_goodness['average correctness'].apply(count_odd)

# Question 5:  Most accurate turks
* We use a mask to filter out the turks with at least 20 rates.
* Then, we list out the mose accurate turks by sort function.

In [8]:
mask_rate_20=rater_goodness['number of ratings']>=20
more_than_20_df=rater_goodness[mask_rate_20]
DataFrame(more_than_20_df.sort_values(by='odds',ascending= False)[:20].index)

,turk
0,A2U0R4X38GUKZE
1,A22C0PJUBFJTI0
2,A23YQUBXZPKILZ
3,ATVALOQVDCMZW
4,A1HIXWH4OXT8S4
5,A3220HG1O83HQ4
6,A32W20KGQXS0LL
7,A20PWAB7G3HDHU
8,AJSJVK40F5HM6
9,A31OCN4MNHUQ6W


# Question 6: Rating counts versus accuracy
* Here we perform a simple linear regression to see if there is a significant relationship.

In [9]:
import statsmodels.api as sm
from patsy import dmatrices

/Users/Jack/anaconda/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [10]:
rater_goodness['num']=rater_goodness['number of ratings']
y, X=dmatrices('odds~num',data= rater_goodness,return_type='dataframe')
model=sm.OLS(y, X)
result=model.fit()
print result.summary()

                            OLS Regression Results                            
Dep. Variable:                   odds   R-squared:                       0.091
Model:                            OLS   Adj. R-squared:                  0.088
Method:                 Least Squares   F-statistic:                     26.77
Date:                Tue, 08 Aug 2017   Prob (F-statistic):           4.52e-07
Time:                        22:37:18   Log-Likelihood:                -2019.6
No. Observations:                 269   AIC:                             4043.
Df Residuals:                     267   BIC:                             4050.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    386.5694     30.441     12.699      0.0

*  As we can see, there is a negative relationship here. This indicates that when turks rate more, they tend to become less accurate.

# Question 7: Overall predicted odds
* First, we create a list of the turks that rated more gold set urls than the other 75% of turks.
* We then merge the rate record with the more than 75% turks of rater to get odds.
* Finally, we groupby url and category to calculate the product of odds.

In [9]:
rater_goodness['turk']=rater_goodness.index
mask_75=rater_goodness['number of ratings']>rater_goodness['number of ratings'].quantile(0.75)
more_75_turks_df=rater_goodness[mask_75]
labels_with_odds=labels_unknown.merge(more_75_turks_df,left_on='turk',right_on='turk')
overall_odds=DataFrame(labels_with_odds.groupby(['url','category'])['odds'].prod())

# Question 8: Predicted categories
* We find out the max odds and category from the dataframe we got in question 7.
* Then, we combine these statistics into one data frame and only extract the columns we want.

In [10]:
top_cat_df=DataFrame(overall_odds.unstack('category')['odds'].T.idxmax())
top_odds_df=DataFrame(overall_odds.unstack('category')['odds'].T.max())
top_cat_df['url']=top_cat_df.index
top_odds_df['url']=top_odds_df.index
cat_odds=top_cat_df.merge(top_odds_df,left_on='url',right_on='url')
cat_odds['top category']=cat_odds['0_x']
cat_odds['top odds']=cat_odds['0_y']
result_75=cat_odds[['url','top category','top odds']]

In [11]:
result_75[:10]

,url,top category,top odds
0,http://0-101.net,G,2.297602
1,http://000.cc,G,1.485456
2,http://0000.jp,G,14.339248
3,http://000relationships.com,G,3.841962
4,http://000vitamins.com,G,4.033641
5,http://000webhost.com,G,11.892639
6,http://003my.com,G,4.861524
7,http://007absolutehardcore.com/early_cumshots/...,X,293.252871
8,http://007swz.cn,G,1.485456
9,http://01768.com,G,1.485456


# Question 9: Predicted categories using more turks
* First, we repeat the process of question 7 and 8, but change the 75% to 25% to get resutl_25.
* Then, we merge them together to get a total result.
* We cross tab the total result to find out where the most errors are.

In [13]:
mask_25=rater_goodness['number of ratings']>rater_goodness['number of ratings'].quantile(0.25)
more_25_turks_df=rater_goodness[mask_25]
labels_with_odds=labels_unknown.merge(more_25_turks_df,left_on='turk',right_on='turk')
overall_odds_25=DataFrame(labels_with_odds.groupby(['url','category'])['odds'].prod())

In [14]:
top_cat_df=DataFrame(overall_odds_25.unstack('category')['odds'].T.idxmax())
top_odds_df=DataFrame(overall_odds_25.unstack('category')['odds'].T.max())
top_cat_df['url']=top_cat_df.index
top_odds_df['url']=top_odds_df.index
cat_odds=top_cat_df.merge(top_odds_df,left_on='url',right_on='url')
cat_odds['top category']=cat_odds['0_x']
cat_odds['top odds']=cat_odds['0_y']
result_25=cat_odds[['url','top category','top odds']]

In [15]:
result_25.index=result_25['url']
result_75.index=result_75['url']

In [16]:
total_result=result_25.merge(result_75,left_on='url',right_on='url',suffixes=['_25','_75'])
error_pd=pd.crosstab(total_result['top category_25'],total_result['top category_75'],margins=True)


In [17]:
error_pd

top category_75,G,P,R,X,All
top category_25,,,,,
G,8280,196,22,28,8526
P,586,352,34,5,977
R,180,52,127,28,387
X,214,25,25,456,720
All,9260,625,208,517,10610


In [18]:
error_pd=pd.crosstab(total_result['top category_25'],total_result['top category_75'],normalize=True)

In [19]:
error_pd*100

top category_75,G,P,R,X
top category_25,,,,
G,78.039585,1.847314,0.207352,0.263902
P,5.523091,3.317625,0.320452,0.047125
R,1.696513,0.490104,1.196984,0.263902
X,2.016965,0.235627,0.235627,4.297832


* Here we can see that the most of difference lies in the category of turk over 25 predicts to be P while turk over 75 predict to be G